In [1]:
import os
%pwd

'd:\\Project\\Machine Learning\\Text Summarization\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Project\\Machine Learning\\Text Summarization'

ENTITY

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    Gradient_accumulation_steps: int

Configuration Manager

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_dirs

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Config_File_Path,
        params_filepath = Params_File_Path
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dirs([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_dirs([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            Gradient_accumulation_steps= params.Gradient_accumulation_steps
        )

        return model_trainer_config
        

Component

In [5]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model= model_pegasus)

        # Loading Dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir= self.config.root_dir,
            num_train_epochs= self.config.num_train_epochs,
            warmup_steps= self.config.warmup_steps,
            per_device_train_batch_size= self.config.per_device_train_batch_size,
            per_device_eval_batch_size= self.config.per_device_train_batch_size,
            weight_decay= self.config.weight_decay,
            logging_steps= self.config.logging_steps,
            evaluation_strategy= self.config.evaluation_strategy,
            eval_steps= self.config.eval_steps,
            save_steps=1e6,
            gradient_accumulation_steps= self.config.Gradient_accumulation_steps
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        # Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

c:\Users\DoomedMean\anaconda3\envs\textSummarize\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-09 10:22:20,092: INFO: config: PyTorch version 2.3.1 available.]


Pipeline

In [6]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e

[2024-07-09 10:22:21,341: INFO: common: yaml file: config\config.yaml Load Successfully]
[2024-07-09 10:22:21,343: INFO: common: yaml file: params.yaml Load Successfully]
[2024-07-09 10:22:21,344: INFO: common: Create directory at artifacts]
[2024-07-09 10:22:21,345: INFO: common: Create directory at artifacts/model_trainer]


TypeError: ModelTrainer() takes no arguments